# Traveling Wave Detection with MCDA

## Required imports

In [1]:
import h5py
import numpy as np
import pandas as pd
from shapely.geometry import LineString

In [2]:
import sys
import os
functions_folder = os.path.abspath('C:/Users/mmill/Documents/STAGE M1/Functions')
if functions_folder not in sys.path:
    sys.path.append(functions_folder)
from Detection_functions import MCDA, circular_mask, preprocessing
from moving_cluster_class import Moving_Cluster

## Choose dataset and associated parameters for detection and classification algorithms

In [3]:
condition='anesthesia' #between anesthesia and awake
session='evokedU' #between evokedU, evokedD and spontaneous

In [4]:
eps=15 if condition=='anesthesia' else 20
min_pts=32
theta=.40 if condition=='anesthesia' else .36 
sign_level=.05 #5%
min_dist = 1 #mm

## Data loading

In [5]:
original_data_directory=r'C:/Users/mmill/Documents/STAGE M1/DATA/'

In [6]:
if condition=='awake':
    if session=='evoked':
        bks_evoked_path = original_data_directory+'gaussian_pos4_ziggy_210908.npy'
        Data = np.load(bks_evoked_path)
    elif session=='spontaneous':
        bks_blank_path = original_data_directory+'blank_ziggy_210908.npy'
        Data = np.load(bks_blank_path)
    mean = np.nanmean(Data)
    nan_loc = np.isnan(Data)
    Data[nan_loc] = mean
    N_trials, Nt, Nx, Ny = Data.shape
    fs = 100 #Hz
    pixel_spacing = 0.045 #mm
    mask = circular_mask((Nx,Ny), center=(219,187), radius=199) 
    border_line=LineString([[47,87] ,[409,131]])

if condition=='anesthesia':
    if session=='evokedD':
        anesth_evokedD_path = original_data_directory+'signal_240306_gaussianD.mat'
        file_anesth_evokedD = h5py.File(anesth_evokedD_path)['signal']
        Data = np.array(file_anesth_evokedD)
    elif session=='evokedU':
        anesth_evokedU_path = original_data_directory+'signal_240306_gaussianU.mat'
        file_anesth_evokedU = h5py.File(anesth_evokedU_path)['signal']
        Data = np.array(file_anesth_evokedU)
    elif session=='spontaneous':
        anesth_blank_path = original_data_directory+'signal_240306_blank.mat' #change it to adapt to your directories
        file_anesth_blank = h5py.File(anesth_blank_path)['signal']
        Data = np.array(file_anesth_blank)
    N_trials, Nt, Nx, Ny = Data.shape
    fs=110 #Hz
    pixel_spacing = 0.064 #mm
    mask = circular_mask((Nx,Ny), center=(145,125), radius=140)
    border_line=LineString([[251,90],[5,118]])

## Run data-collecting algorithms

In [7]:
List_dataframes, mc_number = [], 0
for n in range(N_trials):
    preprocessed_trial = preprocessing(trial=Data[n], border_mask=mask, fs=fs, fc=30, threshold=99)
    detected_mc = MCDA(Trial=preprocessed_trial, Epsilon=eps, MinPts=min_pts, Theta=theta)
    print(f'{len(detected_mc)} moving clusters detected in trial {n}/{N_trials}')
    for mc in detected_mc: 
        moving_cluster=Moving_Cluster(mc['points'], mc['trace'], mc['alpha shapes'], Fs=fs, pixel_spacing=pixel_spacing, condition=condition,session=session, trial=n, number=mc_number)
        moving_cluster.classify(MinDist=min_dist, significance_level=sign_level)
        moving_cluster.propagation_direction(significance_level=sign_level)
        moving_cluster.border_crossing(border_line)
        data = {'MC #':moving_cluster.number, 'Trial #':n, 'Type': moving_cluster.pattern_type,'index':range(moving_cluster.length), 
                'Frames': moving_cluster.trace, 'Time (ms)': moving_cluster.timepoints, 'Duration (ms)':moving_cluster.duration,
                'x_centroid (px)': moving_cluster.centroids[:,0], 'y_centroid (px)': moving_cluster.centroids[:,1], 
                'Translation': moving_cluster.translation, 'Center speed (m/s)': moving_cluster.centroid_speed, 'Propagation direction (rad)': moving_cluster.direction, 
                'Expansion':moving_cluster.expansion, 'Radial speed (m/s)': moving_cluster.radial_speed, 'Surfaces (mm2)': moving_cluster.surfaces, 
                'Border crossing':moving_cluster.border_cross }
        df = pd.DataFrame(data)
        List_dataframes.append(df)
        mc_number+=1
Condition_dataframe = pd.concat(List_dataframes, axis=0, ignore_index=True)
print(f'total of {mc_number} detected moving clusters in this dataset')

5 moving clusters detected in trial 0/20
0 moving clusters detected in trial 1/20
1 moving clusters detected in trial 2/20
0 moving clusters detected in trial 3/20
2 moving clusters detected in trial 4/20
1 moving clusters detected in trial 5/20
5 moving clusters detected in trial 6/20
3 moving clusters detected in trial 7/20
2 moving clusters detected in trial 8/20
3 moving clusters detected in trial 9/20
0 moving clusters detected in trial 10/20
2 moving clusters detected in trial 11/20
2 moving clusters detected in trial 12/20
0 moving clusters detected in trial 13/20
1 moving clusters detected in trial 14/20
3 moving clusters detected in trial 15/20
4 moving clusters detected in trial 16/20
2 moving clusters detected in trial 17/20
1 moving clusters detected in trial 18/20
2 moving clusters detected in trial 19/20
total of 39 detected moving clusters in this dataset


In [8]:
Condition_dataframe.head(21)

,MC #,Trial #,Type,index,Frames,Time (ms),Duration (ms),x_centroid (px),y_centroid (px),Translation,Center speed (m/s),Propagation direction (rad),Expansion,Radial speed (m/s),Surfaces (mm2),Border crossing
0,0,0,Radial,0,5,45.454545,63.636364,166.921053,127.973684,False,NaN,3.631736,True,0.097392,17.960960,True
1,0,0,Radial,1,6,54.545455,63.636364,169.048780,125.408537,False,NaN,3.631736,True,0.045516,28.811264,True
2,0,0,Radial,2,7,63.636364,63.636364,167.561762,122.856069,False,NaN,3.631736,True,0.033880,38.981632,True
3,0,0,Radial,3,8,72.727273,63.636364,166.669818,120.845818,False,NaN,3.631736,True,0.022990,46.227456,True
4,0,0,Radial,4,9,81.818182,63.636364,164.825843,127.169242,False,NaN,3.631736,True,NaN,52.088832,True
5,0,0,Radial,5,10,90.909091,63.636364,163.514053,135.812330,False,NaN,3.631736,True,NaN,50.528256,True
6,0,0,Radial,6,11,100.000000,63.636364,163.491950,136.681574,False,NaN,3.631736,True,NaN,38.936576,True
7,1,0,Static,0,14,127.272727,54.545455,202.434783,128.739130,False,NaN,2.998311,False,NaN,8.644608,True
8,1,0,Static,1,15,136.363636,54.545455,205.264957,116.884615,False,NaN,2.998311,False,NaN,17.041408,True
9,1,0,Static,2,16,145.454545,54.545455,207.242902,115.018927,False,NaN,2.998311,False,NaN,21.207040,True


In [9]:
Condition_dataframe.tail()

,MC #,Trial #,Type,index,Frames,Time (ms),Duration (ms),x_centroid (px),y_centroid (px),Translation,Center speed (m/s),Propagation direction (rad),Expansion,Radial speed (m/s),Surfaces (mm2),Border crossing
325,38,19,Complex,2,27,245.454545,63.636364,83.210526,99.265789,True,0.051178,0.230949,True,0.059386,27.195392,True
326,38,19,Complex,3,28,254.545455,63.636364,84.985989,106.315236,True,0.105285,0.230949,True,NaN,38.580224,True
327,38,19,Complex,4,29,263.636364,63.636364,86.289320,121.213592,True,0.072604,0.230949,True,NaN,34.836480,True
328,38,19,Complex,5,30,272.727273,63.636364,86.007018,131.522807,True,0.014003,0.230949,True,NaN,20.680704,True
329,38,19,Complex,6,31,281.818182,63.636364,84.170000,130.760000,True,NaN,0.230949,True,NaN,8.521728,True


## Data Frame saving

In [10]:
filename=f'{condition.capitalize()}_{session}_bks.xlsx'
print(filename)

Anesthesia_evokedU_bks.xlsx


In [11]:
Condition_dataframe.to_excel(filename)